In [ ]:
import numpy as np
import pandas as pd

## If Running in google colab
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
! ls /content/drive/MyDrive/CSE_272_Final_Project/

chatbot_model.h5		    Generative_Model_Question_Answering.ipynb
chatbot_model_v2.h5		    healthslug.jpg
chatbot_model_v3.h5		    intents.json
chatbot.py			    intents_model.ipynb
checkpoint			    intents_v2.json
classes.pkl			    intents_v3.json
classes_v2.pkl			    IR_Project_Ideas.gdoc
classes_v3.pkl			    models
COVID-Dialogue-Dataset-English.txt  System_Diagram.gslides
Covid-Utterances.csv		    Templated_Reponses.gdoc
Covid-Utterances.gsheet		    words.pkl
Covid-Utterances.xlsx		    words_v2.pkl
CSE_272_Final_Presentation.gslides  words_v3.pkl


In [ ]:
!cp /content/drive/MyDrive/CSE_272_Final_Project/intents_v3.json .

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents_v3.json').read()
lemmatizer = WordNetLemmatizer()
intents = json.loads(data_file, strict=False)



In [ ]:
intents

{'intents': [{'context': [''],
   'patterns': ['Hi there',
    'How are you',
    'Hey',
    'Hola',
    'Hello',
    'Good day',
    "What's up?",
    'Hello!',
    'Greetings',
    'Ayo',
    'Hey hey',
    'Yo',
    'Hi',
    'Hello?',
    'I need help',
    'I need assistance',
    'Help',
    'Help me',
    'howdy',
    'welcome',
    'bonjour',
    'buenas noches',
    'buenos dias',
    'good day',
    'good morning',
    'hi-ya',
    'how are you',
    'how goes it',
    'howdy-do',
    'shalom',
    "what's happening",
    "what's up"],
   'responses': ['Hello, how may I assist you?', 'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'context': [''],
   'patterns': ['Are corona virus and COVID-19 different?',
    'Can I have information about corona virus?',
    'Can you break down the general information for corona virus?',
    'Can you give me information about Covid-19?',
    'Can you tell me about Covid-19?',
    'Can you tell me about the corona virus?',
    'Can yo

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
test_vals = {}
for intent in intents['intents']:
    inds_for_test = np.random.choice(len(intent['patterns']), size = len(intent['patterns'])//5, replace=False)
    inds_for_train = [i for i in np.arange(len(intent['patterns'])) if i not in inds_for_test]
    for i in inds_for_train:
        pattern = intent['patterns'][i]
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
    ls = []
    for i in range(len(inds_for_test)):
        ls.append(intent['patterns'][inds_for_test[i]])
    test_vals[intent['tag']] = ls


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# lemmatize
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# print documents 
print(len(documents), "documents")
# print classes 
print(len(classes), "classes", classes)
# dump words and classes for use later
print(len(words), "lemmatized words", words)
pickle.dump(words,open('words_v3.pkl','wb'))
pickle.dump(classes,open('classes_v3.pkl','wb'))



4928 documents
13 classes ['covid_intro', 'covid_origin', 'current_numbers', 'emergency_contact', 'goodbye', 'greeting', 'medication', 'myth_hoaxes', 'noanswer', 'precautions', 'symptoms_have', 'symptoms_info', 'thanks']
1415 unique lemmatized words ['#', "'", "'5g", "'best", "'conspiracy", "'corona", "'coronavirus", "'covid-19", "'d", "'how", "'m", "'medicine", "'number", "'origin", "'phone", "'precautions", "'preventative", "'s", "'shortness", "'ve", "'what", "'where", "'zoom", '(', ')', '*719', ',', '-19', '.', '0', '1', '10', '12', '19', '2', '2019', '2020', '311', '5g', '6', '60+', '719', '911', ':', 'a', 'abnormally', 'about', 'accept', 'accompanied', 'accurate', 'acetaminophen', 'ache', 'aching', 'action', 'active', 'actually', 'acute', 'address', 'adios', 'administer', 'administered', 'adult', 'advice', 'affect', 'affected', 'affecting', 'africa', 'african', 'after', 'again', 'agains', 'against', 'age', 'aggresive', 'ahave', 'ahve', 'air', 'airborne', 'aiurborne', 'alaska', 'al

In [ ]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
# define model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# train model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'], run_eagerly = True)
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
# dump model
model.save('chatbot_model_v3.h5', hist)
print("model created")


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
986/986 [==============================] - 17s 15ms/step - loss: 1.5975 - accuracy: 0.4825
Epoch 2/200
986/986 [==============================] - 15s 15ms/step - loss: 0.9759 - accuracy: 0.7066
Epoch 3/200
986/986 [==============================] - 15s 15ms/step - loss: 0.7534 - accuracy: 0.7741
Epoch 4/200
986/986 [==============================] - 15s 15ms/step - loss: 0.6491 - accuracy: 0.7908
Epoch 5/200
986/986 [==============================] - 15s 15ms/step - loss: 0.5709 - accuracy: 0.8239
Epoch 6/200
986/986 [==============================] - 15s 15ms/step - loss: 0.5006 - accuracy: 0.8454
Epoch 7/200
986/986 [==============================] - 14s 15ms/step - loss: 0.4590 - accuracy: 0.8573
Epoch 8/200
986/986 [==============================] - 14s 15ms/step - loss: 0.4232 - accuracy: 0.8681
Epoch 9/200
986/986 [==============================] - 15s 15ms/step - loss: 0.3799 - accuracy: 0.8799
Epoch 10/200
986/986 [==============================] - 15s 15ms/step - l

In [ ]:
num_attempted = 0
num_correct = 0
acc_by_key = {}
for key in test_vals.keys():
    num_attempted_key = 0
    num_correct_key = 0
    utterances = test_vals[key]
    for utterance in utterances:
        num_attempted += 1
        num_attempted_key+=1
        if key == predict_class(utterance, model)[0]['intent']:
            num_correct += 1
            num_correct_key += 1
    if num_attempted_key >0:
        acc_by_key[key] = num_correct_key/num_attempted_key

In [ ]:
acc_by_key

{'covid_intro': 0.8958333333333334,
 'covid_origin': 0.8775510204081632,
 'current_numbers': 0.9411764705882353,
 'emergency_contact': 0.9722222222222222,
 'goodbye': 0.5714285714285714,
 'greeting': 1.0,
 'medication': 0.9210526315789473,
 'myth_hoaxes': 0.47368421052631576,
 'precautions': 0.9,
 'symptoms_have': 0.8571428571428571,
 'symptoms_info': 0.84375,
 'thanks': 0.75}

In [ ]:
print('Overall Accuracy: {}'.format(num_correct/num_attempted))
print('Accuracy by key: \n{}'.format(acc_by_key))

Overall Accuracy: 0.8794178794178794
Accuracy by key: 
{'greeting': 1.0, 'covid_intro': 0.8958333333333334, 'covid_origin': 0.8775510204081632, 'symptoms_info': 0.84375, 'symptoms_have': 0.8571428571428571, 'emergency_contact': 0.9722222222222222, 'precautions': 0.9, 'current_numbers': 0.9411764705882353, 'medication': 0.9210526315789473, 'myth_hoaxes': 0.47368421052631576, 'goodbye': 0.5714285714285714, 'thanks': 0.75}


In [ ]:
num_correct

423

In [ ]:
num_attempted

481

In [ ]:
!cp words_v3.pkl /content/drive/MyDrive/CSE_272_Final_Project/
!cp classes_v3.pkl /content/drive/MyDrive/CSE_272_Final_Project/
!cp chatbot_model_v3.h5 /content/drive/MyDrive/CSE_272_Final_Project/